# 集思录转债

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年3月20日

----------

- 研究目的：获取可转债的相关信息；
- 研究方法：从集思录网站爬取可转债相关数据。

In [1]:
#导入库
import numpy as np
import pandas as pd

import json
import os

#引入自定义库
import sys
sys.path.append('/home/jquser')
from ds_jisilu import *

#设置pd显示行列数目
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',100)
pd.set_option('display.width',10000)

#已上市可转债
def get_cb(data_type):
    df=get_jsl_data(data_type)
    #过滤掉可交换债
    df=df[df['btype']=='C']
    #更改列标题
    df=df.rename(columns={
        'bond_id':'代码','bond_nm':'转债名称','price':'现价','increase_rt':'涨跌',
        'stock_id':'正股代码','stock_nm':'正股名称','sprice':'股价','sincrease_rt':'涨跌 ',
        'convert_dt':'转股起始日','convert_price':'转股价','convert_value':'转股价值',
        'premium_rt':'溢价率','convert_amt_ratio':'转债比',
        'force_redeem_price':'强赎触发价','redeem_price_ratio':'强赎触发比',
        'put_price':'回售价',   
        'next_put_dt':'回售起始日','put_convert_price':'回售触发价','put_convert_price_ratio':'回售触发比',             
        'short_maturity_dt':'到期时间','year_left':'剩余(年)','redeem_price':'到期赎回价',    
        'ytm_rt':'税前收益','ytm_rt_tax':'税后收益',           
        'orig_iss_amt':'规模(亿)','curr_iss_amt':'剩余规模(亿)','volume':'成交额(万)','repo_discount_rt':'折算率',           
        'put_tc':'回售条款', 'redeem_tc':'强赎条款','cpn_desc':'利息','adjust_tc':'向下修正',                        
         }) 
    return df

#待发、待上市可转债
def get_cbnew(data_type):
    df=get_jsl_data(data_type)
    #过滤掉可交换债
    df=df[df['cb_type']=='可转债']
    #更改列标题
    df=df.rename(columns={
        'bond_id':'代码','bond_nm':'转债名称',
        'stock_id':'正股代码','stock_nm':'正股名称',
        'amount':'发行规模(亿)',
        'progress_nm':'进展','progress_dt':'进展日期',
        'cb_amount':'百元股票含权(元)',
        'convert_price':'转股价','price':'现价','increase_rt':'涨幅','pma_rt':'现价比转股价',
        'ration_rt':'股东配售率','online_amount':'网上规模(亿)',
        'lucky_draw_rt':'中签率','single_draw':'单账户中签(顶格)','valid_apply':'申购户数(万)',   
         }) 
    #整理进展信息   
    for i in range(len(df)):
        #判断是否是已发
        if df['cp_flag'].iloc[i]=='Y':
            #清除多余字符
            progress=df['进展'].iloc[i].replace('<br>','')
            df['进展日期'].iloc[i]=progress[0:10]
            if progress[10:12]=='上市':
                df['进展'].iloc[i]=progress[10:12]
            else:
                df['进展'].iloc[i]=progress[10:12]+'（'+progress[14:]+'）'
    return df

try:
    cbnew_df=get_cbnew('new_cbond')
    cb_df=get_cb('sell_cbond')
    print(' 转债数据已准备好！')
except Exception as e:
    print(' 转债数据获取失败！')


/opt/conda/envs/python3new/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


 转债数据已准备好！


## 1.已上市转债

### 1.1.基本信息

In [2]:
#基本信息
df=cb_df.loc[:,(
    '代码','转债名称','现价','涨跌',
    '正股名称','股价','涨跌 ',    
    '转股价','转股价值','溢价率',
    '到期时间','剩余(年)',        
    '税前收益','税后收益',           
    '规模(亿)','成交额(万)',
    )]
#按照现价筛选（现价小于100）并排序
df['现价']=df['现价'].astype('float')
df=df[df['现价']<100]
df=df.sort_values('现价',ascending=True)
#重置索引
df=df.reset_index()
del df['index']
df

,代码,转债名称,现价,涨跌,正股名称,股价,涨跌,转股价,转股价值,溢价率,到期时间,剩余(年),税前收益,税后收益,规模(亿),成交额(万)
0,128012,辉丰转债,79.280,1.27%,辉丰股份,2.87,4.74%,7.740,37.080,113.81%,22-04-21,3.775,8.32%,7.93%,8.450,1526.60
1,127003,海印转债,83.969,-0.13%,海印股份,2.43,0.00%,5.250,46.286,81.41%,22-06-08,3.907,8.38%,7.64%,11.110,744.03
2,128013,洪涛转债,84.605,-0.70%,洪涛股份,3.60,1.12%,9.980,36.072,134.54%,22-07-29,4.047,7.58%,6.92%,12.000,964.86
3,128037,岩土转债,85.338,-0.17%,中化岩土,4.99,-0.99%,8.030,62.142,37.33%,24-03-15,5.677,5.19%,4.73%,6.037,187.17
4,128018,时达转债,86.467,0.40%,新时达,6.52,-0.61%,11.830,55.114,56.89%,23-11-06,5.321,4.93%,4.50%,8.825,121.24
5,128026,众兴转债,87.619,-0.16%,众兴菌业,8.50,-2.52%,11.640,73.024,19.99%,23-12-13,5.422,4.61%,4.19%,9.200,174.81
6,128038,利欧转债,87.622,-0.09%,利欧股份,2.06,0.00%,2.750,74.909,16.97%,24-03-22,5.696,4.84%,4.35%,21.975,1307.96
7,113502,嘉澳转债,88.020,-0.03%,嘉澳环保,29.08,1.68%,45.270,64.237,37.02%,23-11-09,5.329,4.96%,4.47%,1.850,132.87
8,123001,蓝标转债,88.055,0.27%,蓝色光标,5.39,-1.10%,9.810,54.944,60.26%,21-12-18,3.436,7.50%,6.77%,14.000,957.50
9,128036,金农转债,88.250,-0.37%,金新农,7.19,-0.69%,9.620,74.740,18.08%,24-03-09,5.660,4.28%,3.87%,6.500,418.84


### 1.2.回售、强赎

In [3]:
#回售、强赎信息
df=cb_df.loc[:,(
    '代码','转债名称','现价','转股价',
    '转股起始日','强赎触发价','强赎触发比',        
    '回售起始日','回售触发价','回售触发比','回售价','到期赎回价',
    '转债比','折算率',       
    )]
#按照现价筛选（现价小于100）并排序
df['现价']=df['现价'].astype('float')
df=df[df['现价']<100]
df=df.sort_values('现价',ascending=True)
#重置索引
df=df.reset_index()
del df['index']
df

,代码,转债名称,现价,转股价,转股起始日,强赎触发价,强赎触发比,回售起始日,回售触发价,回售触发比,回售价,到期赎回价,转债比,折算率
0,128012,辉丰转债,79.280,7.740,2016-10-28,10.06,130.000,2020-04-21,5.42,188.78,103.000,103.000,19.5%,0.00
1,127003,海印转债,83.969,5.250,2016-12-16,6.83,130.000,2020-06-08,3.67,151.23,100.000,110.000,20.3%,0.00
2,128013,洪涛转债,84.605,9.980,2017-02-06,12.97,130.000,2020-07-29,6.99,194.06,100.000,108.000,26.8%,0.26
3,128037,岩土转债,85.338,8.030,2018-09-21,10.44,130.000,2022-03-15,5.62,112.65,100.000,108.000,6.7%,0.00
4,128018,时达转债,86.467,11.830,2018-05-10,15.38,130.000,2021-11-06,8.28,127.01,100.000,106.000,21.8%,0.00
5,128026,众兴转债,87.619,11.640,2018-06-19,15.13,130.000,2021-12-13,8.15,95.86,100.000,106.000,29.0%,0.00
6,128038,利欧转债,87.622,2.750,2018-09-28,3.58,130.000,2022-03-22,1.93,93.45,100.000,109.000,19.1%,0.00
7,113502,嘉澳转债,88.020,45.270,2018-05-17,58.85,130.000,2021-11-10,31.69,108.97,100.000,108.000,8.7%,0.00
8,123001,蓝标转债,88.055,9.810,2016-06-27,12.75,130.000,2019-12-17,6.87,127.40,100.000,108.000,12.1%,0.27
9,128036,金农转债,88.250,9.620,2018-09-16,12.51,130.000,2022-03-09,6.73,93.66,100.000,106.000,23.8%,0.00


### 1.3.条款信息

#### 1.3.1.单条查看

In [5]:
#设置要查看的转债代码
cb_id='128016'
df=cb_df[cb_df['代码']==cb_id]
#重置索引
pd.set_option('max_colwidth', 250)
df.index=df['代码']+' '+df['转债名称']
df.index.name=None
df=df.loc[:,(
    '回售条款', '强赎条款','利息','向下修正',    
    )]
df.T

,128016 雨虹转债
回售条款,（1）有条件回售条款:在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价格的70%时，可转换公司债券持有人有权将其持有的可转换公司债券全部或部分按面值加上当期应计利息的价格回售给公司。若在上述交易日内发生过转股价格因发生送红股、转增股本、增发新股（不包括因本次发行的可转换公司债券转股而增加的股本）、配股以及派发现金股利等情况而调整的情形，则在调整前的交易日按调整前的转股价格和收盘价格计算，在调整后的交易日按调整后的转股价格和收盘价格计算...
强赎条款,"（1）到期赎回条款在本次发行的可转债期满后五个交易日内，公司将以本次可转债票面面值上浮6%（含最后一期利息）的价格向投资者赎回全部未转股的可转债。（2）有条件赎回条款转股期内，当下述两种情形的任意一种出现时，公司有权决定按照债券面值加当期应计利息的价格赎回全部或部分未转股的可转换公司债券：1）在转股期内，如果公司股票在任何连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的130%（含130%）；2）当本次发行的可转换公司债券未转股余额不足3,000万元时。当期应计利息的计算..."
利息,第一年0.3%、第二年0.5%、第三年1.0%、第四年1.3%、第五年1.5%、第六年1.8%
向下修正,在本次发行的可转换公司债券存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收盘价低于当期转股价格的80%时，公司董事会有权提出转股价格向下修正方案并提交公司股东大会表决。上述方案须经出席会议的股东所持表决权的三分之二以上通过方可实施。股东大会进行表决时，持有本次发行的可转换公司债券的股东应当回避。修正后的转股价格应不低于本次股东大会召开日前二十个交易日公司股票交易均价和前一交易日均价之间的较高者。若在前述三十个交易日内发生过转股价格调整的情形，则在转股价格调整日前的交易日...


#### 1.3.2.查看所有

In [6]:
pd.set_option('max_colwidth', 150)
cb_df.index=cb_df['代码']+cb_df['转债名称']
df.index.name=None
df=cb_df.loc[:,(
    '回售条款', '强赎条款','向下修正','利息',  
    )]
df

,回售条款,强赎条款,向下修正,利息
123005万信转债,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价的70%时,在转股期内，如果公司股票在任何连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的130%（含130%）,在本次发行的可转换公司债券存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收盘价低于当期转股价格的85%时,第一年为0.30%、第二年为0.50%、第三年为1.00%、第四年为1.30%、第五年为1.50%、第六年为1.80%。
128038利欧转债,本次发行的可转债最后两个计息年度，如果公司 A 股股票在任何连续 30 个交易日的收盘价格低于当期转股价格的 70%时,在本次发行的可转债转股期内，如果公司 A 股股票连续 30 个交易日中至少有 15 个交易日的收盘价格不低于当期转股价格的 130%（含 130%）,在本次发行的可转债存续期间，当公司 A 股股票在任意连续 30 个交易日中至少有 15 个交易日的收盘价低于当期转股价格的 90%时,第一年 0.3%、第二年 0.5%、第三年 1.0%、第四年 1.5%、第五年 1.8%、第六年 2.0%
123009星源转债,本次发行的可转债最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价格的 70%时,在本次发行的可转债转股期内，如果公司 A 股股票连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的 130%（含 130%）,在本次可转债存续期内，当公司股票在任意二十个连续交易日中至少十个交易日的收盘价格低于当期转股价格 90%时,第一年 0.30%、第二年 0.50%、第三年 1.00%、第四年 1.30%、第五年 1.50%、第六年 1.80%
123008康泰转债,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价的 70%时,在本次发行的可转换公司债券转股期内，如果公司 A 股股票连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的 130%（含130%）,在本次发行的可转换公司债券存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收盘价低于当期转股价格的 85%时,第一年 0.30%、第二年 0.50%、第三年 1.00%、第四年 1.50%、第五年 1.80%、第六年 2.00%
110039宝信转债,在可转债最后两个计息年度内，如果公司 A 股股票任何连续 30 个交易日收盘价格低于当期转股价格的 70%,在转股期内，公司 A 股股票在任何连续 30 个交易日中至少 15 个交易日收盘价格不低于当期转股价格的 130%,在可转债存续期间，当公司 A 股股票在任意 30 个连续交易日中至少 15 个交易日的收盘价格低于当期转股价格的 85%时,本次发行的可转债票面利率第一年为 0.3%、第二年为 0.5%、第三年为 1.0%、第四年为 1.3%、第五年为 1.5%、第六年为 1.8%
113503泰晶转债,本次发行的可转债最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价低于当期转股价格的 70%时,在本次发行的可转债转股期内，如果公司 A 股股票连续三十个交易日中至少有十五个交易日的收盘价不低于当期转股价格的 130%（含）,在本次发行的可转债存续期间，当公司 A 股股票在任意连续二十个交易日中至少有十个交易日的收盘价低于当期转股价格的 90%时,第一年为 0.40%、第二年为 0.60%、第三年为 1.00%、第四年为 1.50%、第五年为 1.80%、第六年为 2.00%。
128036金农转债,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价的 70%时,在本次发行的可转换公司债券转股期内，如果公司 A 股股票连续三十个交易日中至少有十五个交易日的收盘价格不低于当期转股价格的 130%（含130%）,在本次发行的可转换公司债券存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收盘价低于当期转股价格的 90%时,第一年 0.4%、第二年 0.6%、第三年 1.0%、第四年 1.5%、第五年 1.8%、第六年 2.0%
123007道氏转债,在本次发行的可转债最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价的 70%时,在转股期内，如果公司股票在任何连续三十个交易日中至少十五个交易日的收盘价格不低于当期转股价格的 130%（含 130%）,在本次发行的可转债存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收盘价低于当期转股价格的 85%时,第一年为 0.5%，第二年为 0.7%，第三年为 1.0%，第四年为 1.5%，第五年为 1.8%，第六年为 2.0%
128029太阳转债,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续30 个交易日的收盘价格低于当期转股价的 70%时,在本次发行可转换公司债券的转股期内，如果公司 A 股股票连续 30 个交易日中至少有 15 个交易日的收盘价格不低于当期转股价格的 130%（含 130%）,在本次发行的可转换公司债券存续期间，当公司股票在任意连续 30 个交易日中至少有 15 个交易日的收盘价低于当期转股价格的 85%时,第一年 0.3%、第二年 0.5%、第三年 0.8%、第四年 1.0%、第五年 1.5%
127005长证转债,None,转股期内，如果公司股票连续三十个交易日中至少有十五个交易日的收盘价不低于当期转股价格的 130%（含 130%）,在本次发行的可转债存续期间，当公司股票在任意连续三十个交易日中有十五个交易日的收盘价低于当期转股价格的 80%时,第一年 0.2%、第二年 0.4%、第三年 1.0%、第四年 1.5%、第五年 1.8%、第六年 2.0%


## 2.待发、待上市转债

### 2.1.待申购

In [3]:
#筛选待申购    
df=cbnew_df[cbnew_df['ap_flag']=='B']
df=df.loc[:,(
    '正股代码','正股名称',
    '发行规模(亿)','进展','进展日期',
    '百元股票含权(元)','转股价','现价','涨幅','现价比转股价',    
    )] 
#更改列名
df=df.rename(columns={'进展日期':'申购日期','进展':'申购代码'}) 
df['申购代码']=df['申购代码'].str[5:11]
#重置索引
df=df.reset_index()
del df['index']
df

,正股代码,正股名称,发行规模(亿),申购代码,申购日期,百元股票含权(元),转股价,现价,涨幅,现价比转股价
0,603668,天马科技,3.05,754668,2018-04-17,9.47,11.04,10.85,0.09,98.28
1,603421,鼎信通讯,6.00,754421,2018-04-16,6.14,21.86,22.07,-1.25,100.96


### 2.2.待上市

In [4]:
#筛选待上市   
df=cbnew_df[cbnew_df['ap_flag']=='E']
df=df.loc[:,(
    '代码','转债名称',
    '正股名称',
    '进展日期',
    '转股价','现价','涨幅',
    '股东配售率','网上规模(亿)',
    '中签率','单账户中签(顶格)','申购户数(万)', 
    '发行规模(亿)',
    )]
df=df.rename(columns={'进展日期':'上市日期',}) 
#重置索引
df=df.reset_index()
del df['index']
df

,代码,转债名称,正股名称,上市日期,转股价,现价,涨幅,股东配售率,网上规模(亿),中签率,单账户中签(顶格),申购户数(万),发行规模(亿)


### 2.3.已申购

In [5]:
#筛选待申购    
df=cbnew_df[cbnew_df['ap_flag']=='C']
df=df.loc[:,(
    '代码','转债名称',
    '正股名称',
    '进展日期',
    '转股价','现价','涨幅',
    '股东配售率','网上规模(亿)',
    '中签率','单账户中签(顶格)','申购户数(万)', 
    '发行规模(亿)',
    )]
df=df.rename(columns={'进展日期':'申购日期',}) 
#重置索引
df=df.reset_index()
del df['index']
df

,代码,转债名称,正股名称,申购日期,转股价,现价,涨幅,股东配售率,网上规模(亿),中签率,单账户中签(顶格),申购户数(万),发行规模(亿)
0,128038,利欧转债,利欧股份,2018-03-22,2.76,2.66,0.00,26.480,16.16,0.2702,2.7018,59.81,21.98
1,128037,岩土转债,中化岩土,2018-03-15,8.05,7.18,-0.28,13.370,5.89,0.0901,0.9013,65.35,6.80
2,127006,敖东转债,吉林敖东,2018-03-13,21.12,20.62,-0.72,50.900,11.85,0.1238,1.2377,95.75,24.13


### 2.4.待发售

In [6]:
#所有待发售
df=cbnew_df[cbnew_df['ap_flag']=='N']
df=df.loc[:,(
    '正股代码','正股名称',
    '发行规模(亿)','进展','进展日期',
    '百元股票含权(元)','转股价','现价','涨幅','现价比转股价',    
    )] 
#重置索引
df=df.reset_index()
del df['index']
df

,正股代码,正股名称,发行规模(亿),进展,进展日期,百元股票含权(元),转股价,现价,涨幅,现价比转股价
0,002329,皇氏集团,10.80,证监会核准批文,2017-12-21,15.74,6.28,6.08,-1.78,96.82
1,002206,海利得,10.87,证监会核准批文,2018-01-11,13.69,5.11,5.30,-0.38,103.72
2,603027,千禾味业,3.56,证监会核准批文,2018-02-02,5.28,18.53,19.88,-1.49,107.29
3,603345,安井食品,5.00,证监会核准批文,2018-03-12,8.77,26.63,27.91,3.18,104.81
4,300473,德尔股份,6.00,证监会核准批文,2018-03-14,12.18,40.10,41.62,1.04,103.79
5,002224,三力士,6.40,证监会核准批文,2018-03-19,9.68,7.74,8.17,4.08,105.56
6,603955,大千生态,3.90,证监会核准批文,2018-03-20,7.70,33.52,35.88,0.00,107.04
7,603179,新泉股份,4.50,证监会核准批文,2018-03-21,6.12,38.26,40.87,4.05,106.82
8,603588,高能环境,8.40,证监会核准批文,2018-04-11,8.94,11.43,11.34,0.44,99.21
9,603023,威帝股份,2.00,证监会核准批文,2018-04-12,5.48,7.50,7.54,-1.05,100.53
